In [ ]:
# =====================================
# 1. Configuración inicial
# =====================================

# Importamos funciones necesarias de PySpark
from pyspark.sql.functions import col, trim   # col() para referenciar columnas, trim() para limpiar espacios
from pyspark.sql.types import IntegerType, DoubleType  # Para convertir tipos de datos

# Definimos la ruta del archivo en la capa Landing dentro del Lakehouse
landing_path = "Files/landing/sap/persona/clientes.csv"

# Definimos el nombre de la tabla destino en la capa Bronze
bronze_table = "brz_personas"

In [ ]:
# =====================================
# 2. Lectura de datos desde Landing
# =====================================
print(landing_path)
df_landing = (
    spark.read               # Iniciamos la lectura con Spark
    .option("header", True)  # Indicamos que el CSV tiene cabecera
    .option("inferSchema", True)  # Pedimos a Spark que infiera automáticamente los tipos de datos
    .csv(landing_path)       # Especificamos la ruta del archivo en Landing
)

# Mostramos la cantidad total de registros leídos desde Landing
print("Total registros landing:", df_landing.count())

# Mostramos una vista previa de los primeros 5 registros
display(df_landing)

In [ ]:
# =====================================
# 3. Normalización básica para Bronze
# =====================================

df_bronze = (
    df_landing
    # Limpiamos espacios en columnas de texto
    .withColumn("Nombre", trim(col("Nombre")))     
    .withColumn("Apellido", trim(col("Apellido"))) 
    .withColumn("Ciudad", trim(col("Ciudad")))     
    .withColumn("Pais", trim(col("Pais")))         

    # Convertimos campos numéricos a su tipo correcto
    .withColumn("Edad", col("Edad").cast(IntegerType())) 
    .withColumn("FrecuenciaVisita", col("FrecuenciaVisita").cast(IntegerType()))
    .withColumn("TicketPromedio", col("TicketPromedio").cast(DoubleType()))
    .withColumn("PuntosAcumulados", col("PuntosAcumulados").cast(IntegerType()))

    # Aseguramos que los correos estén en texto y sin espacios extra
    .withColumn("Email", trim(col("Email")).cast("string"))
)

# Mostramos una vista previa de los primeros 5 registros ya normalizados
display(df_bronze)

In [ ]:
# =====================================
# 4. Escritura en la capa Bronze
# =====================================

(
    df_bronze.write          # Iniciamos la escritura
    .format("delta")         # Usamos formato Delta para trazabilidad y versionado
    .mode("overwrite")       # Sobrescribimos si la tabla ya existe
    .saveAsTable(bronze_table)  # Guardamos como tabla en la capa Bronze
)

# Confirmamos que la carga fue exitosa
print(f"Datos cargados en Bronze: {bronze_table}")

In [ ]:
# =====================================
# A. Inspección rápida del entorno
# =====================================
# Muestra catálogos disponibles (en Fabric suele usarse 'spark_catalog')
spark.sql("SHOW CATALOGS").show(truncate=False)


# Muestra las bases de datos/esquemas disponibles
spark.sql("SHOW DATABASES").show(truncate=False)

# Muestra la base de datos actual
spark.sql("SELECT current_database()").show()
    